# Finding the minimum number of flips

For every possible initial order of pancakes, we need to know the minimum number of flips needed to sort the stack. We'll now write a function to do so. Our goal in this first pass is to get something that works. We'll optimize later.

## Setup

We'll first grab the helper functions we've already created. If you came up with a different version of `is_sorted` or `flip`, feel free to use yours.

In [ ]:
def is_sorted(pancakes, goal):
    return pancakes == goal

def flip(pancakes, flip_point):
    return pancakes[:flip_point][::-1] + pancakes[flip_point:]

def get_next_states(pancakes):
    for flip_point in range(2, len(pancakes) + 1):
        yield flip(pancakes, flip_point)

We'll need to make use of each of these functions. Let's also define a goal state:

In [ ]:
starting_stack = goal = (1, 2, 3, 4)

And let's also scaffold our new function.

In [ ]:
def find_min_flips(pancakes, goal):
    pass

A nice, easy start. We've blocked out a function called `find_min_flips`. It takes two arguments: 
  - `pancakes`, which will be a tuple representing a starting order
  - `goal`, which will also be a tuple representing the properly sorted order

We haven't implemented the logic yet, but we know what we want the function to return: an integer, the minimum flips required to sort `pancakes`.

## Base Case

A quick win would be nice. Imagine that `pancakes` is already sorted. What would the function look like? Maybe something like this:

In [ ]:
def find_min_flips(pancakes, goal):
    if is_sorted(pancakes, goal):
        return 0

When we're finished, `find_min_flips` will get called once per permutation, and only one of those permutations will be the sorted order. But in that one case, this function would work...

In [ ]:
find_min_flips(starting_stack, goal)

But it's no good in all the other cases...

In [ ]:
find_min_flips((2, 1, 3, 4), goal)

Since an unsorted stack won't trigger the `if` condition in our function, we never execute the return statement. The function completes without encountering a return statement, so it returns `None`. That's why you don't see anything printed when running the code block, above.

Obviously, we still have work to do.

## Basic Strategy

Let's try to map out how this function will get us an answer in all the other (unsorted) cases. If we knew how to find the most efficient flip, our job would be easier (or at least less computationally expensive). But we don't. 

Do we then have to try **every** possible flip? You might think so. After all, we've said again and again that we need to do an *exhaustive* search. But we can do better.

We know (thanks to Bill Gates and others) how to calculate an upper bound for a pancake number. If we're exploring a sequence of flips that grows larger than that upper bound, we can stop. That sequence isn't optimal, so it doesn't interest us. 

But for even fairly small pancake stacks, there are still *a lot* of sequences that are shorter than that upper bound. And in many cases, some fraction of those sequences are both:
  - shorter than the upper bound
  - longer than the optimal sequence

Since they won't get us closer to an answer, it'd be nice to leave those sequences out of our search.

Maybe now is when we have our Eureka! moment: let's look first at the shortest possible sequences. If we can't get to our goal with the shortest possible sequence, we'll next try the second-shortest possible sequences. And then the third-shortest, and the fourth-shortest, and so on until we (finally) find a sequence that results in a sorted stack.

Sound good?

## Searching the Next Shortest Sequences

In retrospect, we can see that our base case "solution" was already starting down this path. The absolute shortest sequence of flips is *no* flips. In the very rare case that our starting order can be reached in zero flips, we're set. 

What would it look like to extend our function to consider not just zero flips, but one flip? Remember that we already wrote a function that can get all the possible states that are one flip away from a given order. 

In [ ]:
def find_min_flips(pancakes, goal):
    if is_sorted(pancakes, goal):
        return 0

    for after_one_flip in get_next_states(pancakes):
        if is_sorted(after_one_flip, goal):
            return 1

`get_next_states` will generate for us all the orders that are possible with just one more flip. Using a `for` loop, we can look at them one at a time and test them using `is_sorted`. As soon as we find one that passes the `is_sorted` test, we can return an answer. There's no point in looking further. We already knew we couldn't get to our goal state in 0 flips and we just discovered a way to get to our goal state in 1 flip. Besides 0 (which we know isn't possible), there's nothing "more minimal" than 1.

Our extended function can now get us an answer in more cases. It still works for our base case (where the stack under consideration is already sorted):

In [ ]:
find_min_flips(starting_stack, goal)

But now it also can get us an answer for an *almost* sorted stack:

In [ ]:
find_min_flips((2, 1, 3, 4), goal)

In [ ]:
find_min_flips((4, 3, 2, 1), goal)

But it's still no help for starting orders that are farther from the goal:

In [ ]:
find_min_flips((3, 4, 2, 1), goal)

## Searching the Next-Next Shortest Sequences??

Maybe we can just keep adding "next-sequences" loops to our function. It'd look something like this:

In [ ]:
def find_min_flips(pancakes, goal):
    if is_sorted(pancakes, goal):
        return 0
    
    for after_one_flip in get_next_states(pancakes):
        if is_sorted(after_one_flip, goal):
            return 1

    for after_one_flip in get_next_states(pancakes):
        for after_two_flips in get_next_states(after_one_flip):
            if is_sorted(after_two_flips, goal):
                return 2

In [ ]:
find_min_flips((3, 4, 2, 1), goal)

And to actually get to the first sorta intresting pancake number (for a stack with three pancakes), we'd need *another* next-sequence loop:

In [ ]:
def find_min_flips(pancakes, goal):
    if is_sorted(pancakes, goal):
        return 0

    for after_one_flip in get_next_states(pancakes):
        if is_sorted(after_one_flip, goal):
            return 1

    for after_one_flip in get_next_states(pancakes):
        for after_two_flips in get_next_states(after_one_flip):
            if is_sorted(after_two_flips, goal):
                return 2

    for after_one_flip in get_next_states(pancakes):
        for after_two_flips in get_next_states(after_one_flip):
            for after_three_flips in get_next_states(after_two_flips):
                if is_sorted(after_three_flips, goal):
                    return 3

In [ ]:
find_min_flips((1, 3, 2), (1, 2, 3))

But that's gross, grossly inefficient, and it just won't scale. We need to find some way to **generalize**.

## Queue it up

Here's a redescription of what that gross code is doing:
 1. We start by testing the single 0-flip order. If it isn't sorted...
 2. We next build a list of all the 1-flip orders and test them one by one. If we find one that is sorted, we're done. If not...
 3. We then build a list of all the 2-flip orders and test them one by one. If we find one that is sorted, we're done. If not...
 4. We build a list of all the 3-flip orders and test them one by one. If we find one that is sorted, we're done. If not...
 5. ...

See a pattern? 

But in our earlier solution, to build the 2-flip list, we had to re-build the 1-flip list. And to build the 3-flip list, we had to re-re-build the 1-flip list and re-build the 2-flip list. That's a lot of wasted work. 

We can do better using a data structure called a ***queue***. A queue is just a list. Items join the back of the queue and "wait" until they get to the front of the queue (and *no cuts*!). You *could* reverse it, but typically, you'd treat the zero-index item as the "front" of the list and the -1-index as the "back" of the list:

```
<front> [item_one, item_two, ... item_last] <back>
```

We add items to the back of the list with `append(<new_item>)` and grab items off the front of the list with `pop(0)`. Here's an example:

In [ ]:
letters = ['a', 'b', 'c', 'd']

# add a letter to the back of the queue
letters.append('e')
letters

In [ ]:
# pop off the letter at the front of the queue
front = letters.pop(0)
front

In [ ]:
letters

Here's how a queue can clean up our code:

In [ ]:
def find_min_flips(pancakes, goal):
    queue = [(pancakes, 0)] # seed the queue with the starting order; each queue item is a tuple: (pancake_stack, num_of_flips)

    while queue: # keep looping as long as the queue has at least one item
        current_pancakes, current_flips = queue.pop(0) # pop off the item at the front of the queue

        if is_sorted(current_pancakes, goal): # test: is the item at the front of the list sorted?
            return current_flips # if so, return the number of flips needed to reach that state

        for next_state in get_next_states(current_pancakes): # generate all the states that are exactly one flip away from the current
            queue.append((next_state, current_flips + 1)) # enqueue each next state, incrementing the flip count by one

    return None # We should never get here! Before the queue is empty, we should have found a sequence that leads to a sorted stack

The queue will keep going, but all the 1-flip orders will be closer to the front of the queue than all the 2-flip orders, which will be closer to the front of the queue than all the 3-flip orders, and so on. It's likely that we'll have some -- and maybe many -- untested states left in the queue. That's okay. The first sequence we find is guaranteed to be the shortest possible sequence, not because we know any nifty tricks, but just because we systematically searched through the possibilities in the right order.

Let's put our new function to the test:

In [ ]:
assert(find_min_flips((1, 2, 3, 4), goal) == 0)

In [ ]:
assert(find_min_flips((3, 2, 1, 4), goal) == 1)

In [ ]:
assert(find_min_flips((2, 3, 1, 4), goal) == 2)

In [ ]:
assert(find_min_flips((1, 3, 2, 4), goal) == 3)

In [ ]:
assert(find_min_flips((2, 4, 1, 3), goal) == 4)

And it'll work for bigger stacks, too...

In [ ]:
assert(find_min_flips((2, 3, 6, 4, 1, 5), (1, 2, 3, 4, 5, 6)) == 5)

Very nice. It's not the most efficient, but it's not bad for a ~first~ ~second~ third(?) pass. And it uses a pattern we'll see again in future search problems.

## Aside: Breadth-First vs. Depth-First

Our inefficient solution that handled up to three flips looked pretty repetitive. We *might* have tried to get rid of the redundancy:

In [ ]:
def find_min_flips(pancakes, goal):
    if is_sorted(pancakes, goal):
        return 0
    
    for after_one_flip in get_next_states(pancakes):
        if is_sorted(after_one_flip, goal):
            return 1
        for after_two_flips in get_next_states(after_one_flip):
            if is_sorted(after_two_flips, goal):
                return 2
            for after_three_flips in get_next_states(after_two_flips):
                if is_sorted(after_three_flips, goal):
                    return 3

It's still not a complete solution, but it *looks* a little cleaner. The problem is, it doesn't consider possibilities in the right order: it might look at a 3-flip sorted order **before** a 1- or 2-flip sorted order. For example,

In [ ]:
find_min_flips((3, 2, 1, 4), goal)

`(3, 2, 1, 4)` is pretty obviously sortable in just one flip, but our "less redundant" reworking of the function tells us it'll take at least 3 flips. Whoops. 

But why? Think about how this code is executed. It considers `(3, 2, 1, 4)` first. It isn't sorted, so it looks at the first flip it can make: `(2, 3, 1, 4)`. Obviously not sorted. But instead of considering the next one-flip order -- `(1, 2, 3, 4)`!! -- it enters a loop that starts looking what results from performing a *second* flip. It's going **deep** first, considering all the possibilities that can be generated from its initial action with a second and third flip. But we want to go **broad** first, considering all the 1-flip possibilities before we bother with any two-flip possibilities.